# Transit Timing Variation (TTV) Analysis

Detect and analyze timing deviations from linear ephemeris.

**Topics:** TTV measurement, O-C diagrams, sinusoidal fitting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from transitkit.core import generate_transit_signal_mandel_agol, add_noise
from transitkit.analysis import measure_transit_timing_variations

## 1. Generate Data with TTVs

Create synthetic data with known timing variations.

In [ ]:
# Parameters
PERIOD = 5.0
DEPTH = 0.01
DURATION = 0.15
TTV_AMPLITUDE = 0.02  # 0.02 days = ~30 minutes
TTV_PERIOD = 50  # Super-period in days

# Long baseline
time = np.linspace(0, 100, 8000)
n_transits = int(100 / PERIOD)

# Generate transits with TTVs
flux = np.ones_like(time)
true_ttvs = []
epochs = []

for n in range(n_transits):
    # Add sinusoidal TTV
    ttv = TTV_AMPLITUDE * np.sin(2 * np.pi * n * PERIOD / TTV_PERIOD)
    t0_actual = PERIOD/2 + n * PERIOD + ttv
    true_ttvs.append(ttv)
    epochs.append(n)
    
    # Generate transit at this time
    transit = generate_transit_signal_mandel_agol(
        time, period=1000, t0=t0_actual, depth=DEPTH, duration=DURATION
    )
    flux = flux * transit

flux = add_noise(flux, noise_level=0.001, seed=42)

print(f'Generated {n_transits} transits with TTV amplitude {TTV_AMPLITUDE*24*60:.1f} minutes')

In [ ]:
# Plot light curve
plt.figure(figsize=(14, 4))
plt.plot(time, flux, 'k.', ms=0.5)
plt.xlabel('Time (days)')
plt.ylabel('Flux')
plt.title('Light Curve with TTVs')
plt.show()

## 2. Measure TTVs

In [ ]:
# Measure TTVs
ttv_result = measure_transit_timing_variations(
    time, flux,
    period=PERIOD,
    t0=PERIOD/2,
    duration=DURATION
)

print('TTV Analysis Results:')
print(f"  TTVs Detected: {ttv_result['ttvs_detected']}")
print(f"  RMS TTV:       {ttv_result['rms_ttv']*24*60:.2f} minutes")
print(f"  p-value:       {ttv_result['p_value']:.4f}")
print(f"  N transits:    {len(ttv_result['ttvs'])}")

## 3. O-C Diagram

In [ ]:
# Plot O-C diagram
measured_epochs = np.array(ttv_result['epochs'])
measured_ttvs = np.array(ttv_result['ttvs'])

fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Measured TTVs
axes[0].plot(measured_epochs, measured_ttvs * 24 * 60, 'bo', ms=6, label='Measured')
axes[0].axhline(0, color='gray', ls='--')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('O-C (minutes)')
axes[0].set_title('Transit Timing Variations (O-C Diagram)')
axes[0].legend()

# Compare with true TTVs (if available)
axes[1].plot(epochs[:len(true_ttvs)], np.array(true_ttvs) * 24 * 60, 'g-', lw=2, label='Injected TTVs')
if len(measured_ttvs) > 0:
    axes[1].plot(measured_epochs, measured_ttvs * 24 * 60, 'ro', ms=4, label='Recovered')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('TTV (minutes)')
axes[1].set_title('Comparison: Injected vs Recovered')
axes[1].legend()

plt.tight_layout()
plt.show()

## 4. TTV Periodogram

In [ ]:
# Simple TTV periodogram using Lomb-Scargle
if len(measured_ttvs) > 5:
    from astropy.timeseries import LombScargle
    
    ls = LombScargle(measured_epochs, measured_ttvs)
    freq, power = ls.autopower(minimum_frequency=0.01, maximum_frequency=0.5)
    ttv_periods = 1 / freq
    
    plt.figure(figsize=(10, 4))
    plt.plot(ttv_periods, power, 'b-')
    plt.axvline(TTV_PERIOD/PERIOD, color='r', ls='--', label=f'True: {TTV_PERIOD/PERIOD:.1f} epochs')
    plt.xlabel('TTV Period (epochs)')
    plt.ylabel('Power')
    plt.title('TTV Periodogram')
    plt.legend()
    plt.xlim(0, 30)
    plt.show()
else:
    print('Not enough transits for periodogram')

## Summary

TTV analysis can reveal:
- **Additional planets**: Gravitational perturbations
- **Planet masses**: Via dynamical modeling
- **Orbital parameters**: Eccentricity, mutual inclination

**Key outputs:**
- `ttvs`: O-C values in days
- `epochs`: Transit epoch numbers
- `rms_ttv`: Scatter in timing
- `p_value`: Statistical significance